### About the experiment

#### Stimulus: 3 or 4 colored dots while fixating

#### Delays: 0.1, 1 or 3 seconds 

#### Task: colored probe stimulus appeared slightly displaced relative to the color-matching cue stimulus. 
 
#### Response: displacement was in the clockwise (CW, response 1) or anticlockwise (CCW, response 0).

#### Number of participants: 8

#### I) Responses are binary -> generalized linear models: to deal with binary/categorical response experiments

#### II) Different delay periods -> interaction factors: to deal with multiple factors of interest and identify an interaction of interest design



In [24]:
# Import data & Libraries
from scipy.stats import *
from scipy.stats import ttest_rel,ttest_1samp,ttest_ind
from scipy.stats import circmean,circvar
from scipy.stats import binned_statistic
from scipy.stats import spearmanr as spear
from scipy.stats.mstats import zscore
from scipy.io import loadmat
from scipy import stats
from numpy import *
import numpy as np
from  numpy import array
from matplotlib.pylab import *
from matplotlib.mlab import *
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core import datetools
from statsmodels.formula.api import ols
from statsmodels.graphics.api import interaction_plot, abline_plot
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.stattools import *
from statsmodels.graphics.gofplots import qqplot
import statsmodels.api as sm
from math import atan2
import sys
from pickle import dump,load
from cmath import phase,polar
import random
import scikits.bootstrap as bootstrap
import patsy
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)


% matplotlib inline


#Functions

leters = "ABDEFGHIJKLMNOPQRSTUVXZ"
jbtest = jarque_bera 

def anovan(y,data,names=leters):
    n_groups = range(len(data[0]))
    model = "y ~ "
    d_data = {}
# import ipdb; ipdb.set_trace()
    d_data["y"] = y
    for g in n_groups:
        model += "C("+names[g]+")*"
        d_data[names[g]] = data[:,g]
    lm = ols(model[:-1], data=pandas.DataFrame(d_data)).fit()
    table = anova_lm(lm)
    resid = lm.resid
    return (table,resid)
#return model,pandas.DataFrame(d_data

def anovan1(y,data,names=leters):
    model = "y ~ "
    d_data = {}
    d_data["y"] = y
    d_data[leters[0]] = data
    lm = ols("y~C("+leters[0]+")", data=pandas.DataFrame(d_data)).fit()
    table = anova_lm(lm)
    resid = lm.resid
    return (table,resid)

def regress(y,data,names=leters):
    n_groups = range(len(data[0]))
    model = "y ~ "
    d_data = {}
    d_data["y"] = y
    for g in n_groups:
        model += names[g]+"+"
        d_data[names[g]] = data[:,g]
    lm = ols(model[:-1], data=pandas.DataFrame(d_data)).fit()
    ci = lm.conf_int()
    params = [lm.params[p] for p in range(len(data[0]))]
    return (params,lm.resid,ci)

def glmfit(y,data):
    n_groups = range(len(data[0]))
    model = "y ~ "
    d_data = {}
    
# import ipdb; ipdb.set_trace()
    d_data["y"] = y
    for g in n_groups:
        model += ""+leters[g]+"+"
        d_data[leters[g]] = data[:,g]
    lm = ols(model[:-1], data=pandas.DataFrame(d_data)).fit()
    ci = m.conf_int()
    return lm

def nans(shape, dtype=float):
    a = empty(shape, dtype)
    a.fill(nan)
    return a

def convert_2pi(angle):
    if angle < 0:
        angle = angle + 2*pi
    return angle

    if angle > pi:
        angle = angle - 2*pi
    return angle


def len2(x):
    if type(x) is not type([]):
        if type(x) is not type(array([])):
            return -1
    return len(x)

def phase2(x):
    if not isnan(x):
        return phase(x)
    return nan

def circdist(angles1,angles2):
    if len2(angles2) < 0:
        if len2(angles1) > 0:
            angles2 = [angles2]*len(angles1)
        else:
            angles2 = [angles2]
            angles1 = [angles1]
    if len2(angles1) < 0:
        angles1 = [angles1]*len(angles2)
    return amap(lambda a1,a2: phase2(exp(1j*a1)/exp(1j*a2)), angles1,angles2)


In [25]:
#Creating the table with variables
vars = ["Response","Probe", "Target", "NonTarget", "Subject", "Load", "Delay"] 
#Defining column names
data = pd.read_csv("all_subjs_exp1.dat", delim_whitespace=True) # Working with data on pandas
data.columns=vars
data['Response'] = data['Response'].map({-1: 0, 1: 1}) #Cleaning binary data, making it 1(clockwise) and 0 ( counterclockwise)


In [26]:
data # shows table matrix 

,Response,Probe,Target,NonTarget,Subject,Load,Delay
0,0.0,5.427974,5.358161,5.166175,1.0,3.0,0.1
1,1.0,2.565634,2.635447,5.044002,1.0,3.0,0.1
2,0.0,5.829400,5.689773,0.558505,1.0,4.0,0.1
3,0.0,1.343904,1.274090,1.047198,1.0,3.0,0.1
4,1.0,1.378810,1.483530,1.221730,1.0,3.0,0.1
5,0.0,3.822271,3.717551,3.944444,1.0,4.0,0.1
6,0.0,4.502949,4.363323,5.253441,1.0,4.0,0.1
7,1.0,1.884956,1.780236,1.989675,1.0,4.0,0.1
8,0.0,4.153884,4.049164,2.321288,1.0,3.0,0.1
9,1.0,5.654867,5.794493,5.515240,1.0,4.0,0.1


In [27]:
dfz=[] #Adding the circular distance between probe and target as another column
dfy=[] # Adding circular distance between target and non target. 

for i in range(len(data)):
    dfz.append(np.asscalar(circdist(data.Probe[i], data.Target[i])))
data['CircdistTP'] = dfz # probe and target

for i in range(len(data)):
    dfy.append(np.asscalar(circdist(data.NonTarget[i], data.Target[i])))

data['CircdistTNT'] = dfy # target and non target
data

,Response,Probe,Target,NonTarget,Subject,Load,Delay,CircdistTP,CircdistTNT
0,0.0,5.427974,5.358161,5.166175,1.0,3.0,0.1,0.069813,-0.191986
1,1.0,2.565634,2.635447,5.044002,1.0,3.0,0.1,-0.069813,2.408554
2,0.0,5.829400,5.689773,0.558505,1.0,4.0,0.1,0.139626,1.151917
3,0.0,1.343904,1.274090,1.047198,1.0,3.0,0.1,0.069813,-0.226893
4,1.0,1.378810,1.483530,1.221730,1.0,3.0,0.1,-0.104720,-0.261799
5,0.0,3.822271,3.717551,3.944444,1.0,4.0,0.1,0.104720,0.226893
6,0.0,4.502949,4.363323,5.253441,1.0,4.0,0.1,0.139626,0.890118
7,1.0,1.884956,1.780236,1.989675,1.0,4.0,0.1,0.104720,0.209440
8,0.0,4.153884,4.049164,2.321288,1.0,3.0,0.1,0.104720,-1.727876
9,1.0,5.654867,5.794493,5.515240,1.0,4.0,0.1,-0.139626,-0.279253


In [28]:
y1, X1 = patsy.dmatrices('Response ~ CircdistTP', data=data, return_type='dataframe')
mod = sm.Logit(y1, X1) #logistic regression the parameter that controls the difficulty of the task.
res = mod.fit()
print res.summary()
# For exercise 1, p value looks significant. So we can say using logstic regression as a parameter is a good choice and the task is difficult. 

Optimization terminated successfully.
         Current function value: 0.609541
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 3502
Model:                          Logit   Df Residuals:                     3500
Method:                           MLE   Df Model:                            1
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1196
Time:                        18:06:26   Log-Likelihood:                -2134.6
converged:                       True   LL-Null:                       -2424.7
                                        LLR p-value:                3.505e-128
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0919      0.037     -2.490      0.013      -0.164      -0.020
CircdistTP    -7.9073      0.

In [29]:
y2, X2 = patsy.dmatrices('Response ~ CircdistTP + CircdistTNT', data=data, return_type='dataframe')
mod = sm.Logit(y2, X2)
res = mod.fit()
print res.summary()

# For exercise 2, P value is >0 so it is not significant for distance between target and non-target.


Optimization terminated successfully.
         Current function value: 0.609515
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 3502
Model:                          Logit   Df Residuals:                     3499
Method:                           MLE   Df Model:                            2
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1197
Time:                        18:06:26   Log-Likelihood:                -2134.5
converged:                       True   LL-Null:                       -2424.7
                                        LLR p-value:                9.665e-127
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0920      0.037     -2.492      0.013      -0.164      -0.020
CircdistTP     -7.9092    

In [30]:
y3, X3 = patsy.dmatrices('Response ~ CircdistTP + CircdistTNT + Delay', data=data, return_type='dataframe')
mod = sm.Logit(y3, X3)
res = mod.fit()
print res.summary()

#For exercise 3, I added additional factor 'delay'. Delay also looks like it doesn't contribute significantly. 
# Could it be related with Logistic regression model? or it could be related with delay period itself. 

Optimization terminated successfully.
         Current function value: 0.609476
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 3502
Model:                          Logit   Df Residuals:                     3498
Method:                           MLE   Df Model:                            3
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1197
Time:                        18:06:26   Log-Likelihood:                -2134.4
converged:                       True   LL-Null:                       -2424.7
                                        LLR p-value:                1.625e-125
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.1137      0.056     -2.044      0.041      -0.223      -0.005
CircdistTP     -7.9099    

In [31]:
df1 = data[data['Delay']<.5] #Checking only delay period of 0.1 sec
y4, X4 = patsy.dmatrices('Response ~ CircdistTP + CircdistTNT', data=df1, return_type='dataframe')
mod = sm.Logit(y4, X4)
res = mod.fit()
print res.summary()

# For exercise 4, the delay period 0.1 sec looks significant! 

Optimization terminated successfully.
         Current function value: 0.612490
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 1165
Model:                          Logit   Df Residuals:                     1162
Method:                           MLE   Df Model:                            2
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1141
Time:                        18:06:27   Log-Likelihood:                -713.55
converged:                       True   LL-Null:                       -805.47
                                        LLR p-value:                 1.200e-40
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.1332      0.064     -2.086      0.037      -0.258      -0.008
CircdistTP     -7.6154    

In [32]:
df2 = data[data['Delay']>.5] #Checking delay periods 1 & 3 secs
y5, X5 = patsy.dmatrices('Response ~ CircdistTP + CircdistTNT', data=df2, return_type='dataframe')
mod = sm.Logit(y5, X5)
res = mod.fit()
print res.summary()

#For exercise 4, the table also shows the delay period for 1 and 3 secs look significant. 
#Delay period didn't give significant result in Exercise 3, so it is better to seperate delay period for different time scales to have proper results! 

Optimization terminated successfully.
         Current function value: 0.606400
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 2337
Model:                          Logit   Df Residuals:                     2334
Method:                           MLE   Df Model:                            2
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1246
Time:                        18:06:27   Log-Likelihood:                -1417.2
converged:                       True   LL-Null:                       -1618.9
                                        LLR p-value:                 2.507e-88
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0696      0.045     -1.535      0.125      -0.158       0.019
CircdistTP     -8.0613    

In [33]:
# For Exercise 5, I did it on pandas instead of using the method in Assignment paper, 
#it seems like I managed to seperate them, but on the table for subjects, it gives the result of nan. 
#Coudl it be related with logical regression model? 
#However by using dir(var) I can check every statistics clearly. 

subjects = np.array(data.Subject) #Creating Dummy variables from subjects categorical data
subjectsmatrix = sm.categorical(subjects)
subjectsdata = pd.DataFrame(subjectsmatrix, columns=('Subs', 'Sub1', 'Sub2', 'Sub3', 'Sub4', 'Sub5', 'Sub6', 'Sub7', 'Sub8'))
datafinal = data.join(subjectsdata)

y6, X6 = patsy.dmatrices('Response ~ CircdistTP + CircdistTNT + Load + Delay + Sub1 + Sub2 + Sub3 + Sub4 + Sub5 + Sub6 + Sub7 + Sub8', data=datafinal, return_type='dataframe')
#Leaving out Subjects(Subs) variable as it is a categorical and not parametric factor, instead I include the dummy variables for all 8 subjects in binary
mod = sm.Logit(y6, X6)
res = mod.fit()
print res.summary()


Optimization terminated successfully.
         Current function value: 0.608273
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:               Response   No. Observations:                 3502
Model:                          Logit   Df Residuals:                     3490
Method:                           MLE   Df Model:                           11
Date:                Sun, 22 Apr 2018   Pseudo R-squ.:                  0.1215
Time:                        18:06:27   Log-Likelihood:                -2130.2
converged:                       True   LL-Null:                       -2424.7
                                        LLR p-value:                3.124e-119
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0403        nan        nan        nan         nan         nan
CircdistTP     -7.9337    

C:\Users\gizem\Anaconda2\lib\site-packages\statsmodels\base\model.py:1029: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self.cov_params()))
C:\Users\gizem\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\gizem\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\gizem\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [34]:
dir(mod) #to check each of the available statistics

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_perfect_pred',
 '_data_attr',
 '_derivative_exog',
 '_derivative_predict',
 '_get_init_kwds',
 '_handle_data',
 '_init_keys',
 'cdf',
 'cov_params_func_l1',
 'data',
 'df_model',
 'df_resid',
 'endog',
 'endog_names',
 'exog',
 'exog_names',
 'fit',
 'fit_regularized',
 'from_formula',
 'hessian',
 'information',
 'initialize',
 'jac',
 'k_constant',
 'loglike',
 'loglikeobs',
 'pdf',
 'predict',
 'raise_on_perfect_prediction',
 'score',
 'score_obs']